In [1]:
spark

In [2]:
data=!echo $SPARK_HOME/examples/src/main/resources/employees.json

In [3]:
ipt_json=spark.read.format("json").load(data[0])

In [4]:
ipt_json.show()

+-------+------+
|   name|salary|
+-------+------+
|Michael|  3000|
|   Andy|  4500|
| Justin|  3500|
|  Berta|  4000|
+-------+------+



In [12]:
#writing to delta

In [5]:
ipt_json.write.format("delta").save("/home/somanath/delata_opt")

In [13]:
#fetching data plan


In [5]:
delta_read=spark.read.format("delta").load("/home/somanath/delata_opt")

In [6]:
delta_read.count()

4

In [20]:
spark.sql("set -v").toPandas().to_csv("spark_2.4_propertis.csv")

In [22]:
delta_read.rdd.getNumPartitions()

1

# ensure durability will not allow schema change

In [25]:
delta_read=delta_read.withColumn("test_col",delta_read.salary*2)

In [43]:
#catching exception
try :
    delta_read.write.format("delta").mode("append").save("/home/somanath/delata_opt")
except Exception as e:
    print(str(e))

u'A schema mismatch detected when writing to the Delta table.\nTo enable schema migration, please set:\n\'.option("mergeSchema", "true")\'.\n\nTable schema:\nroot\n-- name: string (nullable = true)\n-- salary: long (nullable = true)\n\n\nData schema:\nroot\n-- name: string (nullable = true)\n-- salary: long (nullable = true)\n-- test_col: long (nullable = true)\n\n         \nIf Table ACLs are enabled, these options will be ignored. Please use the ALTER TABLE\ncommand for changing the schema.\n        ;'



# maintaining history with spark delta

In [10]:
delta_read=delta_read.selectExpr("*","salary+2000 as salary_new").drop("salary").selectExpr("name","salary_new as salary")

In [11]:
delta_read.write.format("delta").mode("append").save("/home/somanath/delata_opt")

## reading current version

In [12]:
delta_read_new=spark.read.format("delta").load("/home/somanath/delata_opt")

In [13]:
delta_read_new.show()

+-------+------+
|   name|salary|
+-------+------+
|Michael|  5000|
|   Andy|  6500|
| Justin|  5500|
|  Berta|  6000|
|Michael|  3000|
|   Andy|  4500|
| Justin|  3500|
|  Berta|  4000|
+-------+------+



## reading old version

In [28]:
#getting version no from json
!ls -lt /home/somanath/delata_opt/_delta_log/

total 8
-rw-r--r-- 1 somanath somanath 308 Sep 28 11:27 00000000000000000001.json
-rw-r--r-- 1 somanath somanath 729 Sep 28 08:07 00000000000000000000.json


In [34]:
cat /home/somanath/delata_opt/_delta_log/00000000000000000001.json

{"commitInfo":{"timestamp":1569650235323,"operation":"WRITE","operationParameters":{"mode":"Append","partitionBy":"[]"},"readVersion":0}}
{"add":{"path":"part-00000-f5772fc5-946d-48fe-98b9-81e094154fa6-c000.snappy.parquet","partitionValues":{},"size":683,"modificationTime":1569650235000,"dataChange":true}}


In [25]:
delta_read_old=spark.read.format("delta").option("versionAsOf","0").load("/home/somanath/delata_opt")

In [26]:
delta_read_old.show()

+-------+------+
|   name|salary|
+-------+------+
|Michael|  3000|
|   Andy|  4500|
| Justin|  3500|
|  Berta|  4000|
+-------+------+



# checking history and schema  with  overwrite mode

In [30]:
delta_read=delta_read.withColumn("test_col",delta_read.salary*2)
try :
    delta_read.write.format("delta").mode("overwrite").save("/home/somanath/delata_opt")
except Exception as e:
    print(str(e))

u'A schema mismatch detected when writing to the Delta table.\nTo enable schema migration, please set:\n\'.option("mergeSchema", "true")\'.\n\nTable schema:\nroot\n-- name: string (nullable = true)\n-- salary: long (nullable = true)\n\n\nData schema:\nroot\n-- name: string (nullable = true)\n-- salary: long (nullable = true)\n-- test_col: long (nullable = true)\n\n         \nTo overwrite your schema or change partitioning, please set:\n\'.option("overwriteSchema", "true")\'.\n\nNote that the schema can\'t be overwritten when using\n\'replaceWhere\'.\n         \nIf Table ACLs are enabled, these options will be ignored. Please use the ALTER TABLE\ncommand for changing the schema.\n        ;'


In [31]:
delta_read=delta_read.selectExpr("*","salary+2000 as salary_new").drop("salary").selectExpr("name","salary_new as salary")
delta_read.write.format("delta").mode("overwrite").save("/home/somanath/delata_opt")

In [32]:
delta_read.show()

+-------+------+
|   name|salary|
+-------+------+
|Michael| 13000|
|   Andy| 14500|
| Justin| 13500|
|  Berta| 14000|
|Michael| 11000|
|   Andy| 12500|
| Justin| 11500|
|  Berta| 12000|
+-------+------+



## reading earlier version data

In [35]:
#getting version no from json
!ls -lt /home/somanath/delata_opt/_delta_log/

total 12
-rw-r--r-- 1 somanath somanath 765 Sep 28 11:38 00000000000000000002.json
-rw-r--r-- 1 somanath somanath 308 Sep 28 11:27 00000000000000000001.json
-rw-r--r-- 1 somanath somanath 729 Sep 28 08:07 00000000000000000000.json


In [37]:

!cat  /home/somanath/delata_opt/_delta_log/00000000000000000002.json

{"commitInfo":{"timestamp":1569650896977,"operation":"WRITE","operationParameters":{"mode":"Overwrite","partitionBy":"[]"},"readVersion":1}}
{"add":{"path":"part-00000-2affc6fc-934b-4e68-8fea-a8024fc9bad0-c000.snappy.parquet","partitionValues":{},"size":683,"modificationTime":1569650896000,"dataChange":true}}
{"add":{"path":"part-00001-3e594723-4254-441a-8b25-43e7619b681b-c000.snappy.parquet","partitionValues":{},"size":683,"modificationTime":1569650896000,"dataChange":true}}
{"remove":{"path":"part-00000-f5772fc5-946d-48fe-98b9-81e094154fa6-c000.snappy.parquet","deletionTimestamp":1569650896976,"dataChange":true}}
{"remove":{"path":"part-00000-ab96a785-1efc-407a-8f8f-38a59cbde32c-c000.snappy.parquet","deletionTimestamp":1569650896977,"dataChange":true}}


In [40]:
!cat  /home/somanath/delata_opt/_delta_log/00000000000000000001.json

{"commitInfo":{"timestamp":1569650235323,"operation":"WRITE","operationParameters":{"mode":"Append","partitionBy":"[]"},"readVersion":0}}
{"add":{"path":"part-00000-f5772fc5-946d-48fe-98b9-81e094154fa6-c000.snappy.parquet","partitionValues":{},"size":683,"modificationTime":1569650235000,"dataChange":true}}


In [41]:
!cat  /home/somanath/delata_opt/_delta_log/00000000000000000000.json

{"commitInfo":{"timestamp":1569638251431,"operation":"WRITE","operationParameters":{"mode":"ErrorIfExists","partitionBy":"[]"}}}
{"protocol":{"minReaderVersion":1,"minWriterVersion":2}}
{"metaData":{"id":"097a59ca-cd52-47c3-b4bc-2798afca43b6","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"name\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"salary\",\"type\":\"long\",\"nullable\":true,\"metadata\":{}}]}","partitionColumns":[],"configuration":{},"createdTime":1569638246364}}
{"add":{"path":"part-00000-ab96a785-1efc-407a-8f8f-38a59cbde32c-c000.snappy.parquet","partitionValues":{},"size":683,"modificationTime":1569638251000,"dataChange":true}}


In [43]:
#matches the opt with cell 32
delta_read_new=spark.read.format("delta").load("/home/somanath/delata_opt")
delta_read_new.show()

+-------+------+
|   name|salary|
+-------+------+
|Michael|  9000|
|   Andy| 10500|
| Justin|  9500|
|  Berta| 10000|
|Michael|  7000|
|   Andy|  8500|
| Justin|  7500|
|  Berta|  8000|
+-------+------+



In [45]:
delta_read_old=spark.read.format("delta").option("versionAsOf","0").load("/home/somanath/delata_opt")
delta_read_old.show()

+-------+------+
|   name|salary|
+-------+------+
|Michael|  3000|
|   Andy|  4500|
| Justin|  3500|
|  Berta|  4000|
+-------+------+



In [47]:
delta_read_old=spark.read.format("delta").option("timestampAsOf","2019-09-28 08:30:00.000").load("/home/somanath/delata_opt")
delta_read_old.show()

+-------+------+
|   name|salary|
+-------+------+
|Michael|  3000|
|   Andy|  4500|
| Justin|  3500|
|  Berta|  4000|
+-------+------+



In [48]:
##merge operations will be done using scala api